In [2]:
import torch
import pandas as pd
import numpy as np
import random

In [3]:
data = pd.read_csv('dataset_train.csv')

In [4]:
data = data.fillna(0)

In [5]:
train = data.drop(labels='Index',axis=1)

In [6]:
x_train = train.iloc[:,1:]
y_train = train.iloc[:, 0]

In [7]:
df = pd.get_dummies(x_train["Best Hand"], prefix="Hand")

In [8]:
x_train = x_train.join(df)

In [9]:
x_train["Birthday"] = pd.to_datetime(x_train["Birthday"])

In [10]:
start =  pd.Timestamp("1970-01-01")

In [11]:
start

Timestamp('1970-01-01 00:00:00')

In [12]:
x_train["Birthday"] = (x_train["Birthday"] - start)//pd.Timedelta('1s')

In [13]:
x_train = x_train.drop(labels='Best Hand',axis = 1)

In [15]:
def to_cat(strs):
    Alpha = {i:j for j,i in enumerate('ABCDEFGHIJKLMNOPQRSTUVWXYZ')}
    add = 0
    for s in strs:
        add+=Alpha[s.upper()]
    return round(add/len(strs),2)

In [16]:
encoding = {i:j for j,i in enumerate(y_train.unique())}
def encode(strs):
    return encoding[strs]

In [17]:
y_train = y_train.apply(encode)

In [18]:
x_train["First Name"] = x_train["First Name"].apply(to_cat)

In [19]:
x_train["Last Name"] = x_train["Last Name"].apply(to_cat)

In [20]:
x_train.head()

,First Name,Last Name,Birthday,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying,Hand_Left,Hand_Right
0,8.00,15.00,954374400,58384.0,-487.886086,5.727180,4.878861,4.722,272.035831,532.484226,5.231058,1039.788281,3.790369,0.715939,-232.79405,-26.89,1,0
1,7.60,8.71,939859200,67239.0,-552.060507,-5.987446,5.520605,-5.612,-487.340557,367.760303,4.107170,1058.944592,7.248742,0.091674,-252.18425,-113.45,0,1
2,12.50,10.20,941587200,23702.0,-366.076117,7.725017,3.660761,6.140,664.893521,602.585284,3.555579,1088.088348,8.728531,-0.515327,-227.34265,30.42,1,0
3,12.40,6.44,966643200,32667.0,697.742809,-6.497214,-6.977428,4.026,-537.001128,523.982133,-4.809637,920.391449,0.821911,-0.014040,-256.84675,200.64,1,0
4,11.67,6.80,906854400,60158.0,436.775204,-7.820623,0.000000,2.236,-444.262537,599.324514,-3.444377,937.434724,4.311066,-0.264070,-256.38730,157.98,1,0


In [44]:
from sklearn.ensemble import RandomForestClassifier

In [45]:
forest = RandomForestClassifier(n_estimators=200,n_jobs=-1)

In [49]:
y_trains = pd.get_dummies(data.iloc[:, 1])

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_trains, test_size=0.1,random_state=42)

In [73]:
forest.fit(x_train,y_train)

RandomForestClassifier(n_estimators=200, n_jobs=-1)

In [74]:
from sklearn.metrics import classification_report

In [53]:
d_test = pd.read_csv('dataset_test.csv')

,Index,Hogwarts House,First Name,Last Name,Birthday,Best Hand,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying
0,0,NaN,Rico,Sargent,2001-10-06,Right,41642.0,696.096071,3.020172,-6.960961,7.996,-365.151850,393.138185,4.207691,1046.742736,3.668983,0.373853,-244.48172,-13.62
1,1,NaN,Tamara,Shackelford,1998-01-08,Left,45352.0,-370.844655,2.965226,3.708447,6.349,522.580486,602.853051,6.460017,1048.053878,8.514622,0.577432,-231.29200,-26.26
2,2,NaN,Staci,Crandall,1998-09-15,Left,43502.0,320.303990,-6.185697,-3.203040,4.619,-630.073207,588.071795,-5.565818,936.437358,1.850829,-1.647150,-252.99343,200.15
3,3,NaN,Dee,Gavin,2001-05-10,Right,61831.0,407.202928,4.962442,NaN,NaN,-449.179806,427.699966,NaN,1043.397718,4.656573,1.164708,-244.01660,-11.15
4,4,NaN,Gregory,Gustafson,1999-02-01,Right,NaN,288.337747,3.737656,-2.883377,4.886,-449.732166,385.712782,2.876347,1051.377936,2.750586,0.102104,-243.99806,-7.12


In [55]:
test = d_test.drop(labels='Index',axis=1)
test = test.drop(labels='Hogwarts House', axis=1)

In [58]:
df = pd.get_dummies(d_test["Best Hand"], prefix="Hand")
test = test.join(df)

,First Name,Last Name,Birthday,Best Hand,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying,Hand_Left,Hand_Right
0,Rico,Sargent,2001-10-06,Right,41642.0,696.096071,3.020172,-6.960961,7.996,-365.151850,393.138185,4.207691,1046.742736,3.668983,0.373853,-244.48172,-13.62,0,1
1,Tamara,Shackelford,1998-01-08,Left,45352.0,-370.844655,2.965226,3.708447,6.349,522.580486,602.853051,6.460017,1048.053878,8.514622,0.577432,-231.29200,-26.26,1,0
2,Staci,Crandall,1998-09-15,Left,43502.0,320.303990,-6.185697,-3.203040,4.619,-630.073207,588.071795,-5.565818,936.437358,1.850829,-1.647150,-252.99343,200.15,1,0
3,Dee,Gavin,2001-05-10,Right,61831.0,407.202928,4.962442,NaN,NaN,-449.179806,427.699966,NaN,1043.397718,4.656573,1.164708,-244.01660,-11.15,0,1
4,Gregory,Gustafson,1999-02-01,Right,NaN,288.337747,3.737656,-2.883377,4.886,-449.732166,385.712782,2.876347,1051.377936,2.750586,0.102104,-243.99806,-7.12,0,1


In [59]:
test = test.drop(labels='Best Hand',axis=1)

In [61]:
test["Birthday"] = pd.to_datetime(test["Birthday"])

In [62]:
test["First Name"] = test["First Name"].apply(to_cat)

In [63]:
test["Last Name"] = test["Last Name"].apply(to_cat)

In [65]:
test = test.fillna(0)

In [67]:
test["Birthday"] = (test["Birthday"] - start)//pd.Timedelta('1s')

In [68]:
test.head()

,First Name,Last Name,Birthday,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying,Hand_Left,Hand_Right
0,10.25,11.00,1002326400,41642.0,696.096071,3.020172,-6.960961,7.996,-365.151850,393.138185,4.207691,1046.742736,3.668983,0.373853,-244.48172,-13.62,0,1
1,8.00,8.27,884217600,45352.0,-370.844655,2.965226,3.708447,6.349,522.580486,602.853051,6.460017,1048.053878,8.514622,0.577432,-231.29200,-26.26,1,0
2,9.40,7.12,905817600,43502.0,320.303990,-6.185697,-3.203040,4.619,-630.073207,588.071795,-5.565818,936.437358,1.850829,-1.647150,-252.99343,200.15,1,0
3,3.67,9.60,989452800,61831.0,407.202928,4.962442,0.000000,0.000,-449.179806,427.699966,0.000000,1043.397718,4.656573,1.164708,-244.01660,-11.15,0,1
4,12.57,12.56,917827200,0.0,288.337747,3.737656,-2.883377,4.886,-449.732166,385.712782,2.876347,1051.377936,2.750586,0.102104,-243.99806,-7.12,0,1


In [80]:
print(classification_report(y_val,forest.predict(x_val),target_names=y_train.columns))

              precision    recall  f1-score   support

  Gryffindor       0.97      1.00      0.98        29
  Hufflepuff       0.98      0.98      0.98        54
   Ravenclaw       1.00      0.96      0.98        52
   Slytherin       0.96      1.00      0.98        25

   micro avg       0.98      0.98      0.98       160
   macro avg       0.98      0.99      0.98       160
weighted avg       0.98      0.98      0.98       160
 samples avg       0.98      0.98      0.98       160



In [81]:
y_test = forest.predict(test)

In [85]:
encodes = []
for i in y_test:
    for j in range(len(y_train.columns)):
        if i[j] == 1:
            encodes.append(y_train.columns[j])

['Hufflepuff',
 'Ravenclaw',
 'Gryffindor',
 'Hufflepuff',
 'Hufflepuff',
 'Slytherin',
 'Ravenclaw',
 'Hufflepuff',
 'Ravenclaw',
 'Hufflepuff',
 'Hufflepuff',
 'Slytherin',
 'Slytherin',
 'Slytherin',
 'Slytherin',
 'Hufflepuff',
 'Ravenclaw',
 'Slytherin',
 'Ravenclaw',
 'Hufflepuff',
 'Hufflepuff',
 'Slytherin',
 'Ravenclaw',
 'Ravenclaw',
 'Hufflepuff',
 'Hufflepuff',
 'Ravenclaw',
 'Slytherin',
 'Gryffindor',
 'Ravenclaw',
 'Hufflepuff',
 'Hufflepuff',
 'Gryffindor',
 'Hufflepuff',
 'Hufflepuff',
 'Ravenclaw',
 'Gryffindor',
 'Ravenclaw',
 'Slytherin',
 'Ravenclaw',
 'Ravenclaw',
 'Slytherin',
 'Hufflepuff',
 'Slytherin',
 'Hufflepuff',
 'Ravenclaw',
 'Slytherin',
 'Gryffindor',
 'Ravenclaw',
 'Slytherin',
 'Ravenclaw',
 'Hufflepuff',
 'Ravenclaw',
 'Ravenclaw',
 'Hufflepuff',
 'Slytherin',
 'Slytherin',
 'Ravenclaw',
 'Hufflepuff',
 'Gryffindor',
 'Gryffindor',
 'Gryffindor',
 'Hufflepuff',
 'Hufflepuff',
 'Ravenclaw',
 'Hufflepuff',
 'Hufflepuff',
 'Ravenclaw',
 'Hufflepuff',
 

In [86]:
test["House"] = encodes

In [87]:
test.head()

,First Name,Last Name,Birthday,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying,Hand_Left,Hand_Right,House
0,10.25,11.00,1002326400,41642.0,696.096071,3.020172,-6.960961,7.996,-365.151850,393.138185,4.207691,1046.742736,3.668983,0.373853,-244.48172,-13.62,0,1,Hufflepuff
1,8.00,8.27,884217600,45352.0,-370.844655,2.965226,3.708447,6.349,522.580486,602.853051,6.460017,1048.053878,8.514622,0.577432,-231.29200,-26.26,1,0,Ravenclaw
2,9.40,7.12,905817600,43502.0,320.303990,-6.185697,-3.203040,4.619,-630.073207,588.071795,-5.565818,936.437358,1.850829,-1.647150,-252.99343,200.15,1,0,Gryffindor
3,3.67,9.60,989452800,61831.0,407.202928,4.962442,0.000000,0.000,-449.179806,427.699966,0.000000,1043.397718,4.656573,1.164708,-244.01660,-11.15,0,1,Hufflepuff
4,12.57,12.56,917827200,0.0,288.337747,3.737656,-2.883377,4.886,-449.732166,385.712782,2.876347,1051.377936,2.750586,0.102104,-243.99806,-7.12,0,1,Hufflepuff


In [21]:
def create_batches():
    batches = []
    for i in range(1600):
        batches.append((torch.from_numpy(x_train.iloc[i].values.astype('float32')),
                      torch.from_numpy(np.array(y_train[i].astype('float32')))))
    return batches

In [22]:
y_train[0].astype('float32')

0.0

In [23]:
batches = create_batches()

In [24]:
y_train.unique()

array([0, 1, 2, 3], dtype=int64)

In [25]:
len(batches)

1600

In [26]:
train_dl, val_dl = torch.utils.data.random_split(batches, [1400, 200])

In [27]:
train_dl = torch.utils.data.DataLoader(train_dl,batch_size=100,shuffle=True)

In [28]:
val_dl = torch.utils.data.DataLoader(val_dl, batch_size=200)

In [29]:
len(val_dl)

1

In [30]:
y_train.shape

(1600,)

In [31]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [32]:
input_size = 18
num_classes = 4

In [38]:
class HogModel(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 6),
            torch.nn.ReLU(),
            torch.nn.Linear(6, num_classes)
        )
        
    def forward(self, xb):
        xb = xb.view(-1, input_size)
        out = self.model(xb)
        return out
    
    def train_step(self, batch):
        cols, label = batch
        out = self(cols)
        loss = torch.nn.functional.cross_entropy(out, label.long())
        return loss
    
    def val_step(self, batch):
        cols, labels = batch 
        out = self(cols)                    # Generate predictions
        loss = torch.nn.functional.cross_entropy(out, labels.long())   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
    
model = HogModel()

In [40]:
def evaluate(model, val_loader):
    outputs = [model.val_step(batch) for batch in val_loader]
    return model.validation_epoch(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.train_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [41]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1155, -0.1918,  0.1165,  0.1567, -0.2032,  0.2067, -0.1706, -0.1706,
          -0.1533,  0.2145,  0.1913,  0.0383,  0.1248, -0.0110,  0.1407,  0.0620,
          -0.1562, -0.1983],
         [-0.0402,  0.1153, -0.1842,  0.1560,  0.0955,  0.1417,  0.0179, -0.1267,
          -0.1238, -0.0146, -0.0602,  0.1641,  0.1115,  0.0375,  0.2159, -0.1760,
           0.1282, -0.1720],
         [-0.1833, -0.1232,  0.1697,  0.0686,  0.0882,  0.0258, -0.1290, -0.0917,
          -0.0915, -0.0793,  0.1299, -0.1283, -0.1577,  0.0185, -0.0193, -0.1805,
           0.0529,  0.1188],
         [ 0.0789,  0.0472,  0.0324,  0.0323,  0.2316, -0.2116,  0.0903, -0.1738,
           0.0044,  0.1783,  0.0407, -0.1556,  0.0471,  0.1738, -0.0336,  0.0698,
          -0.0861, -0.0179],
         [-0.2008, -0.1980,  0.1677, -0.2336, -0.1442,  0.2157, -0.0073,  0.0244,
           0.0134, -0.1265, -0.0288, -0.0605, -0.0179, -0.1381, -0.1776, -0.0679,
           0.1042,  0.1916],
         [ 0

In [42]:
history = []
history+=fit(100,0.01,model,train_dl, val_dl)

Epoch [0], val_loss: 1.3778, val_acc: 0.3200
Epoch [1], val_loss: 1.3708, val_acc: 0.3200
Epoch [2], val_loss: 1.3644, val_acc: 0.3200
Epoch [3], val_loss: 1.3600, val_acc: 0.3200
Epoch [4], val_loss: 1.3573, val_acc: 0.3200
Epoch [5], val_loss: 1.3558, val_acc: 0.3200
Epoch [6], val_loss: 1.3546, val_acc: 0.3200
Epoch [7], val_loss: 1.3543, val_acc: 0.3200
Epoch [8], val_loss: 1.3540, val_acc: 0.3200
Epoch [9], val_loss: 1.3534, val_acc: 0.3200
Epoch [10], val_loss: 1.3534, val_acc: 0.3200
Epoch [11], val_loss: 1.3533, val_acc: 0.3200
Epoch [12], val_loss: 1.3535, val_acc: 0.3200
Epoch [13], val_loss: 1.3532, val_acc: 0.3200
Epoch [14], val_loss: 1.3538, val_acc: 0.3200
Epoch [15], val_loss: 1.3539, val_acc: 0.3200
Epoch [16], val_loss: 1.3533, val_acc: 0.3200
Epoch [17], val_loss: 1.3536, val_acc: 0.3200
Epoch [18], val_loss: 1.3538, val_acc: 0.3200
Epoch [19], val_loss: 1.3533, val_acc: 0.3200
Epoch [20], val_loss: 1.3535, val_acc: 0.3200
Epoch [21], val_loss: 1.3535, val_acc: 0.320